# Building GPT From Scratch

## 1. Data Loading and Preparation

### Reading the Text Data

In [12]:
import torch

In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

### Inspecting the Data Size

In [2]:
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


### Previewing the Dataset

In [5]:
print(text[:100])# print the first 100 characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


## 2. Tokenization

### Creating the Vocabulary
A neural network can't process raw characters hence it needs numbers. Therefore our first step is to create a "vocabulary" of all the unique characters present in our text. We also calculate the `vocab_size` which is the number of unique characters. This will determine the size of our model's embedding and output layers.

In [7]:
chars = sorted(list(set(text)))# create a sorted list of unique characters
vocab_size = len(chars)
print("All the unique characters in the dataset are as follows: ", ''.join(chars))

All the unique characters in the dataset are as follows:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
print("Vocabulary size:", vocab_size)

Vocabulary size: 65


### Creating Encoder and Decoder

Now we create a mapping (or "tokenizer") that can convert characters to integers and back.
- **`stoi` (string-to-integer):** A dictionary that maps each unique character to a unique integer.
- **`itos` (integer-to-string):** A dictionary that does the reverse, mapping integers back to characters.
- `encode`: A function that takes a string and converts it into a list of integers (tokens) using stoi.
- `decode`: A function that takes a list of integers and converts it back into a string using itos.

In [10]:
# create a mapping from characters to integers and vice versa
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [11]:
print(encode("hello world")) # encode the string "hello world" to integers
print(decode(encode("hello world"))) # decode back to string

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


### Encoding the Entire Dataset

In [13]:
data = torch.tensor(encode(text), dtype=torch.long)# encode the entire text dataset and store it in a torch tensor
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## 3. Data Splitting and Batching

### Train and Validation Split


In [14]:
n = int(0.9*len(data))# first 90% will be train, rest validation
train_data = data[:n]
val_data = data[n:]

### Understanding Context and Targets
A language model works by predicting the next token in a sequence given a preceding context. The maximum length of this context is called the **block size** or **context length**.<br>
Now we perform two tasks:
- The input `x` is a chunk of text of length `block_size`.
- The target `y` is the same chunk of text but shifted one position to the right.

For every position `t` in the sequence the model will learn to predict the target `y[t]` using the context of all tokens up to `x[t]`.

In [15]:
block_size = 8# context length for predictions
train_data[:block_size+1]# a chunk of text of length block_size + 1

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x = train_data[:block_size]# input
y = train_data[1:block_size+1]# target
for t in range(block_size):
    context = x[:t+1]# the context is everything up to and including position t
    target = y[t]# the target is the next character we want to predict
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


### Creating a Batch Generation Function
Training on the entire dataset at once is computationally expensive. Instead we train on small random chunks of data called minibatches.
This function `get_batch` does the following:
1. Selects `batch_size` (e.g., 4) random starting points in the dataset (`train_data` or `val_data`).
2. For each starting point it grabs a sequence of length `block_size` for the input (`x`).
3. It grabs the corresponding sequence of length block_size (shifted by one) for the target (y).
4. It stacks these individual sequences into two tensors `xb` and `yb` of shape (batch_size, block_size).

This provides a batch of independent examples that we can process in parallel which is highly efficient on modern hardware like GPUs.

In [18]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))# pick random starting indices for the batch 
    x = torch.stack([data[i:i+block_size] for i in ix])# gather the input sequences for each starting index
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])# gather the target sequences (shifted by one) for each starting index
    return x, y


xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [20]:
for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, :t+1]# the context is everything up to and including position t
        target = yb[b, t]# the target is the next character we want to predict
        print(f"When input is {context.tolist()} the target: {target}")

When input is [24] the target: 43
When input is [24, 43] the target: 58
When input is [24, 43, 58] the target: 5
When input is [24, 43, 58, 5] the target: 57
When input is [24, 43, 58, 5, 57] the target: 1
When input is [24, 43, 58, 5, 57, 1] the target: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
When input is [44] the target: 53
When input is [44, 53] the target: 56
When input is [44, 53, 56] the target: 1
When input is [44, 53, 56, 1] the target: 58
When input is [44, 53, 56, 1, 58] the target: 46
When input is [44, 53, 56, 1, 58, 46] the target: 39
When input is [44, 53, 56, 1, 58, 46, 39] the target: 58
When input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
When input is [52] the target: 58
When input is [52, 58] the target: 1
When input is [52, 58, 1] the target: 58
When input is [52, 58, 1, 58] the target: 46
When input is [52, 58, 1, 58, 46] the target: 39
When input is [52, 58, 1, 58, 46, 39] the t